In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [3]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['DHI','DNI','WS','RH','T']])
df_test[['DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['DHI','DNI','WS','RH','T']])

In [6]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

In [8]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [9]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
model7 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [12]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=300, batch_size=48, validation_split=0.25)
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')]
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1323 - val_loss: 1.8148
Epoch 2/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0931 - val_loss: 1.8409
Epoch 3/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0815 - val_loss: 1.7829
Epoch 4/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1056 - val_loss: 1.7852
Epoch 5/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0890 - val_loss: 1.7666
Epoch 6/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0981 - val_loss: 1.8267
Epoch 7/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1030 - val_loss: 1.8359
Epoch 8/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1088 - val_loss: 1.7909
Epoch 9/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0924 - val_loss: 1.8278
Epoch 10/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0883 - val_lo

820/820 [==============================] - 1s 1ms/step - loss: 1.0663 - val_loss: 1.7972
Epoch 72/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0965 - val_loss: 1.8181
Epoch 73/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0760 - val_loss: 1.8373
Epoch 74/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0661 - val_loss: 1.8565
Epoch 75/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0602 - val_loss: 1.8198
Epoch 76/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0423 - val_loss: 1.8979
Epoch 77/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0825 - val_loss: 1.8206
Epoch 78/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0647 - val_loss: 1.7948
Epoch 79/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0713 - val_loss: 1.9148
Epoch 80/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0707 - val_loss: 

Epoch 151/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0469 - val_loss: 1.9023
Epoch 152/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0363 - val_loss: 1.8717
Epoch 153/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0513 - val_loss: 1.8246
Epoch 154/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0257 - val_loss: 1.8376
Epoch 155/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0430 - val_loss: 1.9135
Epoch 156/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0330 - val_loss: 1.9102
Epoch 157/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0244 - val_loss: 1.8266
Epoch 158/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0555 - val_loss: 1.9204
Epoch 159/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0347 - val_loss: 1.8534
Epoch 160/300
820/820 [==============================] - 1s 1ms/step - lo

820/820 [==============================] - 1s 1ms/step - loss: 1.0430 - val_loss: 1.8574
Epoch 231/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0167 - val_loss: 1.8777
Epoch 232/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0145 - val_loss: 1.8857
Epoch 233/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0350 - val_loss: 1.8589
Epoch 234/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0254 - val_loss: 1.8663
Epoch 235/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0179 - val_loss: 1.8436
Epoch 236/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0310 - val_loss: 1.9897
Epoch 237/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0072 - val_loss: 1.8863
Epoch 238/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0122 - val_loss: 1.8440
Epoch 239/300
820/820 [==============================] - 1s 1ms/step - loss: 1.0207 - v

820/820 [==============================] - 1s 1ms/step - loss: 1.7059 - val_loss: 2.8371
Epoch 2/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6888 - val_loss: 2.8732
Epoch 3/300
820/820 [==============================] - 1s 1ms/step - loss: 1.7025 - val_loss: 2.8622
Epoch 4/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6566 - val_loss: 2.8355
Epoch 5/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6855 - val_loss: 2.8586
Epoch 6/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6529 - val_loss: 2.8184
Epoch 7/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6637 - val_loss: 2.9061
Epoch 8/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6742 - val_loss: 2.8685
Epoch 9/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6385 - val_loss: 2.8637
Epoch 10/300
820/820 [==============================] - 1s 1ms/step - loss: 1.6656 - val_loss: 2.8183
E

...

820/820 [==============================] - 1s 1ms/step - loss: 0.6744 - val_loss: 1.3549
Epoch 36/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6758 - val_loss: 1.2680
Epoch 37/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6782 - val_loss: 1.2519
Epoch 38/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6834 - val_loss: 1.3120
Epoch 39/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6765 - val_loss: 1.2905
Epoch 40/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6664 - val_loss: 1.2123
Epoch 41/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6621 - val_loss: 1.3666
Epoch 42/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6725 - val_loss: 1.4208
Epoch 43/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6741 - val_loss: 1.2758
Epoch 44/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6800 - val_loss: 

820/820 [==============================] - 1s 1ms/step - loss: 0.6505 - val_loss: 1.2351
Epoch 116/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6653 - val_loss: 1.2875
Epoch 117/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6594 - val_loss: 1.2369
Epoch 118/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6475 - val_loss: 1.1775
Epoch 119/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6455 - val_loss: 1.2557
Epoch 120/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6644 - val_loss: 1.1869
Epoch 121/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6547 - val_loss: 1.2717
Epoch 122/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6568 - val_loss: 1.2563
Epoch 123/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6555 - val_loss: 1.2098
Epoch 124/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6637 - v

820/820 [==============================] - 1s 1ms/step - loss: 0.6520 - val_loss: 1.1890
Epoch 195/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6463 - val_loss: 1.2537
Epoch 196/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6463 - val_loss: 1.2517
Epoch 197/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6464 - val_loss: 1.2850
Epoch 198/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6482 - val_loss: 1.1586
Epoch 199/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6453 - val_loss: 1.2207
Epoch 200/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6416 - val_loss: 1.2282
Epoch 201/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6381 - val_loss: 1.2710
Epoch 202/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6522 - val_loss: 1.2705
Epoch 203/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6510 - v

820/820 [==============================] - 1s 1ms/step - loss: 0.6510 - val_loss: 1.2463
Epoch 274/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6428 - val_loss: 1.2139
Epoch 275/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6272 - val_loss: 1.2759
Epoch 276/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6388 - val_loss: 1.1737
Epoch 277/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6341 - val_loss: 1.2905
Epoch 278/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6442 - val_loss: 1.2324
Epoch 279/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6313 - val_loss: 1.3431
Epoch 280/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6516 - val_loss: 1.2046
Epoch 281/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6514 - val_loss: 1.1762
Epoch 282/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6270 - v

(3888, 9)

In [13]:
model8 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [14]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=300, batch_size=48, validation_split=0.25)
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')]
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4910 - val_loss: 1.6623
Epoch 2/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4373 - val_loss: 1.6842
Epoch 3/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4525 - val_loss: 1.6775
Epoch 4/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4277 - val_loss: 1.6262
Epoch 5/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4288 - val_loss: 1.6290
Epoch 6/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4381 - val_loss: 1.6171
Epoch 7/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4276 - val_loss: 1.6275
Epoch 8/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4110 - val_loss: 1.6078
Epoch 9/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4122 - val_loss: 1.6036
Epoch 10/300
820/820 [==============================] - 1s 1ms/step - loss: 1.4308 - val_lo

820/820 [==============================] - 1s 1ms/step - loss: 1.3344 - val_loss: 1.5854
Epoch 72/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3259 - val_loss: 1.5523
Epoch 73/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3237 - val_loss: 1.5708
Epoch 74/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3151 - val_loss: 1.6378
Epoch 75/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3231 - val_loss: 1.5878
Epoch 76/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3072 - val_loss: 1.5917
Epoch 77/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3144 - val_loss: 1.6517
Epoch 78/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3311 - val_loss: 1.5881
Epoch 79/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3173 - val_loss: 1.5740
Epoch 80/300
820/820 [==============================] - 1s 1ms/step - loss: 1.3224 - val_loss: 

Epoch 151/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2691 - val_loss: 1.6731
Epoch 152/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2533 - val_loss: 1.6456
Epoch 153/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2570 - val_loss: 1.6112
Epoch 154/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2444 - val_loss: 1.6335
Epoch 155/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2790 - val_loss: 1.6121
Epoch 156/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2808 - val_loss: 1.6223
Epoch 157/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2473 - val_loss: 1.6614
Epoch 158/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2547 - val_loss: 1.6019
Epoch 159/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2409 - val_loss: 1.6040
Epoch 160/300
820/820 [==============================] - 1s 1ms/step - lo

820/820 [==============================] - 1s 1ms/step - loss: 1.2290 - val_loss: 1.6728
Epoch 231/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2098 - val_loss: 1.6450
Epoch 232/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2077 - val_loss: 1.6306
Epoch 233/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2004 - val_loss: 1.6701
Epoch 234/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2090 - val_loss: 1.6894
Epoch 235/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2049 - val_loss: 1.6765
Epoch 236/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2002 - val_loss: 1.6597
Epoch 237/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1915 - val_loss: 1.6747
Epoch 238/300
820/820 [==============================] - 1s 1ms/step - loss: 1.2118 - val_loss: 1.6675
Epoch 239/300
820/820 [==============================] - 1s 1ms/step - loss: 1.1914 - v

820/820 [==============================] - 1s 1ms/step - loss: 2.0509 - val_loss: 2.6920
Epoch 2/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9799 - val_loss: 2.6895
Epoch 3/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9506 - val_loss: 2.6879
Epoch 4/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9793 - val_loss: 2.7062
Epoch 5/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9610 - val_loss: 2.7171
Epoch 6/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9756 - val_loss: 2.7063
Epoch 7/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9739 - val_loss: 2.7076
Epoch 8/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9503 - val_loss: 2.6849
Epoch 9/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9680 - val_loss: 2.6808
Epoch 10/300
820/820 [==============================] - 1s 1ms/step - loss: 1.9754 - val_loss: 2.6693
E

...

820/820 [==============================] - 1s 1ms/step - loss: 0.7085 - val_loss: 1.2635
Epoch 35/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7138 - val_loss: 1.2832
Epoch 36/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7127 - val_loss: 1.3568
Epoch 37/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6986 - val_loss: 1.2857
Epoch 38/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7136 - val_loss: 1.2897
Epoch 39/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6975 - val_loss: 1.3090
Epoch 40/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7072 - val_loss: 1.3262
Epoch 41/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7112 - val_loss: 1.3119
Epoch 42/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7002 - val_loss: 1.2800
Epoch 43/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7162 - val_loss: 

820/820 [==============================] - 1s 1ms/step - loss: 0.7007 - val_loss: 1.2655
Epoch 115/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6968 - val_loss: 1.2513
Epoch 116/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6908 - val_loss: 1.2611
Epoch 117/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6968 - val_loss: 1.3232
Epoch 118/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6996 - val_loss: 1.3270
Epoch 119/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7032 - val_loss: 1.3183
Epoch 120/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6981 - val_loss: 1.3036
Epoch 121/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7005 - val_loss: 1.2380
Epoch 122/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6985 - val_loss: 1.3175
Epoch 123/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6866 - v

820/820 [==============================] - 1s 1ms/step - loss: 0.6864 - val_loss: 1.2994
Epoch 194/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6927 - val_loss: 1.2724
Epoch 195/300
820/820 [==============================] - 1s 1ms/step - loss: 0.7045 - val_loss: 1.3428
Epoch 196/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6919 - val_loss: 1.2910
Epoch 197/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6749 - val_loss: 1.2667
Epoch 198/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6903 - val_loss: 1.2811
Epoch 199/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6941 - val_loss: 1.3000
Epoch 200/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6763 - val_loss: 1.3146
Epoch 201/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6880 - val_loss: 1.3268
Epoch 202/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6926 - v

820/820 [==============================] - 1s 1ms/step - loss: 0.6899 - val_loss: 1.2665
Epoch 273/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6869 - val_loss: 1.2652
Epoch 274/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6956 - val_loss: 1.3050
Epoch 275/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6942 - val_loss: 1.2868
Epoch 276/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6849 - val_loss: 1.2497
Epoch 277/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6917 - val_loss: 1.2353
Epoch 278/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6920 - val_loss: 1.2944
Epoch 279/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6742 - val_loss: 1.2626
Epoch 280/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6904 - val_loss: 1.2858
Epoch 281/300
820/820 [==============================] - 1s 1ms/step - loss: 0.6862 - v

(3888, 9)

In [15]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.001216,-0.001011,0.000091,0.000018,0.000527,0.000317,0.000725,-0.000054,-0.000038
1,0.csv_Day7_0h30m,-0.001216,-0.001011,0.000091,0.000018,0.000527,0.000317,0.000725,-0.000054,-0.000038
2,0.csv_Day7_1h00m,-0.001216,-0.001011,0.000091,0.000018,0.000527,0.000317,0.000725,-0.000054,-0.000038
3,0.csv_Day7_1h30m,-0.001216,-0.001011,0.000091,0.000018,0.000527,0.000317,0.000725,-0.000054,-0.000038
4,0.csv_Day7_2h00m,-0.001216,-0.001011,0.000091,0.000018,0.000527,0.000317,0.000725,-0.000054,-0.000038
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,-0.009258,-0.013139,-0.002060,0.004957,0.002932,0.003696,-0.000703,0.004737,-0.001106
7772,80.csv_Day8_22h00m,-0.009662,-0.013711,-0.002021,0.005173,0.003219,0.003888,-0.000971,0.004250,-0.000106
7773,80.csv_Day8_22h30m,-0.009797,-0.013707,-0.002031,0.005173,0.003204,0.003844,-0.000896,0.004378,-0.000242
7774,80.csv_Day8_23h00m,-0.010197,-0.014274,-0.001987,0.005402,0.003455,0.004036,-0.001172,0.003905,0.000797


In [16]:
submission.to_csv('./submission/submission_20210113-2.csv', index=False)